In [ ]:
# ==============================================================================
# 1. INSTALACIÓN Y CONFIGURACIÓN INICIAL
# ==============================================================================
# Instalar bibliotecas necesarias
!pip install streamlit joblib pandas numpy plotly scikit-learn lightgbm xgboost

# Instalar ngrok y pyngrok para visualizar Streamlit en Colab
!pip install pyngrok

# Importar bibliotecas
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import joblib
import sys
import os
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import warnings
from pyngrok import ngrok
import subprocess
import time

warnings.filterwarnings('ignore')

# ==============================================================================
# 2. GESTIÓN DE RUTAS Y ESTRUCTURA DE CARPETAS PARA COLAB
# ==============================================================================
# Definir la ruta raíz del proyecto para Colab
def get_project_root():
    """Obtener la ruta raíz del proyecto: el directorio actual en Colab."""
    return os.getcwd()

PROJECT_ROOT = get_project_root()

# Crear la estructura de directorios necesaria
os.makedirs(os.path.join(PROJECT_ROOT, 'data', 'raw'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, 'models'), exist_ok=True)

print(f"Carpeta de datos creada en: {os.path.join(PROJECT_ROOT, 'data', 'raw')}")
print("¡Recuerda subir tus CSV de la NASA a esta carpeta!")

# ==============================================================================
# 3. DEFINICIÓN DE CLASES Y FUNCIONES (Backend Puro)
# ==============================================================================

class ExoplanetDataProcessor:
    """
    Procesador de datos para los datasets reales de la NASA.
    (Backend puro, NO usa st.info, st.error, etc.)
    """

    def __init__(self):
        self.scaler = StandardScaler()
        self.feature_names = []

    def load_real_data(self):
        """Cargar los datasets reales de la NASA con rutas corregidas"""
        try:
            data_dir = os.path.join(PROJECT_ROOT, 'data', 'raw')

            print(f"🔍 Buscando datos en: {data_dir}")

            if not os.path.exists(data_dir):
                print(f"❌ No existe el directorio: {data_dir}")
                return None, None, None

            files = os.listdir(data_dir)
            print(f"📁 Archivos encontrados en data/raw/: {files}")

            # Construir rutas completas
            kepler_path = os.path.join(data_dir, 'kepler.csv')
            k2_path = os.path.join(data_dir, 'k2.csv')
            tess_path = os.path.join(data_dir, 'tess.csv')

            print(f"📊 Intentando cargar:\n- {kepler_path}\n- {k2_path}\n- {tess_path}")

            if not os.path.exists(kepler_path):
                print(f"❌ No existe: {kepler_path}. Por favor, súbelo.")
                return None, None, None

            # Cargar los archivos reales
            kepler_df = pd.read_csv(kepler_path)
            k2_df = pd.read_csv(k2_path) if os.path.exists(k2_path) else None
            tess_df = pd.read_csv(tess_path) if os.path.exists(tess_path) else None

            print(f"✅ Kepler cargado: {len(kepler_df)} registros")
            if k2_df is not None:
                print(f"✅ K2 cargado: {len(k2_df)} registros")
            if tess_df is not None:
                print(f"✅ TESS cargado: {len(tess_df)} registros")

            return kepler_df, k2_df, tess_df

        except Exception as e:
            print(f"❌ Error cargando datasets: {e}")
            return None, None, None

    def preprocess_kepler(self, df):
        """Preprocesar datos Kepler reales"""
        df_clean = df.copy()

        if 'koi_disposition' not in df_clean.columns:
            print("❌ No se encuentra la columna 'koi_disposition' en Kepler")
            return None

        # Eliminar columnas no útiles
        columns_to_drop = ['kepid', 'kepoi_name', 'kepler_name', 'koi_pdisposition', 'koi_score']
        columns_to_drop = [col for col in columns_to_drop if col in df_clean.columns]
        if columns_to_drop:
            df_clean = df_clean.drop(columns=columns_to_drop)

        # Filtrar solo confirmed, candidate y false positive
        valid_dispositions = ['CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE']
        mask = df_clean['koi_disposition'].isin(valid_dispositions)
        df_clean = df_clean[mask]

        # Crear target binario
        df_clean['target'] = df_clean['koi_disposition'].map({
            'CONFIRMED': 1,
            'CANDIDATE': 1,
            'FALSE POSITIVE': 0
        })

        # Añadir identificador de misión
        df_clean['mission'] = 'kepler'

        print(f"✅ Kepler procesado: {len(df_clean)} registros válidos")

        return df_clean

    def preprocess_k2(self, df):
        """Preprocesar datos K2 reales"""
        if df is None: return None

        df_clean = df.copy()
        if 'disposition' not in df_clean.columns:
            print("❌ No se encuentra la columna 'disposition' en K2")
            return None

        df_clean = df_clean[df_clean['disposition'].isin(['CONFIRMED', 'CANDIDATE'])]

        # Target binario
        df_clean['target'] = df_clean['disposition'].map({
            'CONFIRMED': 1,
            'CANDIDATE': 1
        })
        df_clean['mission'] = 'k2'
        print(f"✅ K2 procesado: {len(df_clean)} registros válidos")
        return df_clean

    def preprocess_tess(self, df):
        """Preprocesar datos TESS reales"""
        if df is None: return None

        df_clean = df.copy()
        if 'tfopwg_disp' not in df_clean.columns:
            print("❌ No se encuentra la columna 'tfopwg_disp' en TESS")
            return None

        # Mapear disposiciones de TESS
        disposition_mapping = {
            'PC': 1, 'KP': 1, 'APC': 1,  # Positivos
            'FP': 0, 'FA': 0  # Negativos
        }

        df_clean['target'] = df_clean['tfopwg_disp'].map(disposition_mapping)
        df_clean = df_clean.dropna(subset=['target'])
        df_clean['mission'] = 'tess'
        print(f"✅ TESS procesado: {len(df_clean)} registros válidos")
        return df_clean

    def prepare_features(self, df):
        """Preparar características para el modelo - VERSIÓN FLEXIBLE"""
        if df is None or len(df) == 0:
            print("❌ No hay datos para preparar características")
            return None, None, None

        # Posibles nombres de columnas en diferentes datasets
        possible_features = {
            'orbital_period': ['koi_period', 'pl_orbper', 'period'],
            'transit_duration': ['koi_duration', 'pl_trandurh', 'duration'],
            'transit_depth': ['koi_depth', 'pl_trandep', 'depth'],
            'planet_radius': ['koi_prad', 'pl_rade', 'radius'],
            'equilibrium_temp': ['koi_teq', 'pl_eqt', 'teq'],
            'insolation_flux': ['koi_insol', 'pl_insol', 'insol'],
            'stellar_teff': ['koi_steff', 'st_teff', 'teff'],
            'stellar_logg': ['koi_slogg', 'st_logg', 'logg'],
            'stellar_radius': ['koi_srad', 'st_rad', 'srad']
        }

        # Encontrar las columnas disponibles
        available_columns = []
        for feature_name, possible_names in possible_features.items():
            for name in possible_names:
                if name in df.columns:
                    available_columns.append(name)
                    break

        print(f"📊 Columnas numéricas encontradas: {available_columns}")

        if not available_columns:
            print("❌ No se encontraron columnas numéricas para entrenar")
            return None, None, None

        X = df[available_columns].copy()
        y = df['target'].values

        # Manejar valores missing
        X = X.fillna(X.median())

        # Escalar características: fit_transform para entrenamiento
        X_scaled = self.scaler.fit_transform(X)
        self.feature_names = available_columns # Guardar los nombres de las features
        print(f"✅ Características preparadas: {X_scaled.shape}")

        return X_scaled, y, available_columns

class RealExoplanetModel:
    """
    Modelo real para entrenamiento con datos de la NASA.
    (Backend puro, NO usa st.info, st.error, etc.)
    """
    def __init__(self):
        self.model = None
        self.accuracy = 0
        self.feature_importance = None

    def create_ensemble(self):
        """Crear ensemble con los algoritmos del paper"""
        base_models = [
            ('random_forest', RandomForestClassifier(
                n_estimators=100, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1
            )),
            ('extra_trees', ExtraTreesClassifier(
                n_estimators=100, max_depth=10, random_state=42, n_jobs=-1
            )),
            ('xgboost', XGBClassifier(
                n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42, use_label_encoder=False, eval_metric='logloss'
            )),
            ('lightgbm', LGBMClassifier(
                n_estimators=100, learning_rate=0.05, max_depth=6, random_state=42, n_jobs=-1, verbose=-1 # Silenciar LightGBM
            ))
        ]

        ensemble = StackingClassifier(
            estimators=base_models,
            final_estimator=LogisticRegression(),
            cv=3,
            passthrough=False,
            n_jobs=-1
        )
        return ensemble

    def train(self, X, y):
        """Entrenar el modelo real"""
        if X is None or y is None:
            print("❌ No hay datos para entrenar")
            return None

        print("🤖 Iniciando entrenamiento del ensemble...")

        self.model = self.create_ensemble()
        self.model.fit(X, y)

        # Calcular accuracy en entrenamiento
        y_pred = self.model.predict(X)
        self.accuracy = accuracy_score(y, y_pred)

        print(f"📈 Accuracy en entrenamiento: {self.accuracy:.2%}")

        # Calcular importancia de características
        self._calculate_feature_importance(X.shape[1])

        return self.model

    def _calculate_feature_importance(self, n_features):
        """Calcular importancia de características promediada"""
        importances = np.zeros(n_features)
        count = 0

        for name, model in self.model.named_estimators_.items():
            if hasattr(model, 'feature_importances_'):
                importances += model.feature_importances_
                count += 1

        if count > 0:
            self.feature_importance = importances / count
        else:
            self.feature_importance = None

    def save_model(self, filepath):
        """Guardar modelo entrenado"""
        if self.model:
            os.makedirs(os.path.dirname(filepath), exist_ok=True)
            joblib.dump(self.model, filepath)
            return True
        return False

    def load_model(self, filepath):
        """Cargar modelo entrenado"""
        try:
            if os.path.exists(filepath):
                self.model = joblib.load(filepath)
                self.accuracy = 0
                return True
        except Exception as e:
            print(f"Error cargando modelo: {e}")
        return False

# ==============================================================================
# 4. CLASE PRINCIPAL DE STREAMLIT (Frontend)
# ==============================================================================

class ExoplanetDetectorApp:
    def __init__(self):
        self.model = RealExoplanetModel()
        self.data_processor = ExoplanetDataProcessor()
        self.model_trained = False

        self.load_initial_state()

    def load_initial_state(self):
        """Cargar modelo y preprocesador guardados si existen"""
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        processor_path = os.path.join(PROJECT_ROOT, 'models', 'data_processor.pkl')
        
        # Carga del modelo
        if os.path.exists(model_path):
            if self.model.load_model(model_path):
                self.model_trained = True

        # Carga del procesador (que contiene scaler y feature_names)
        if os.path.exists(processor_path):
            try:
                # El procesador cargado YA contiene el .scaler y .feature_names
                self.data_processor = joblib.load(processor_path)
            except Exception as e:
                pass # Si falla la carga, usa el procesador recién inicializado

    def render_sidebar(self):
        """Barra lateral de navegación"""
        st.sidebar.title("🔭 NASA Exoplanet Detector - REAL")
        st.sidebar.markdown("---")

        page = st.sidebar.radio("Navegación", [
            "🏠 Inicio",
            "🚀 Entrenar Modelo REAL",
            "🤖 Clasificar Exoplanetas",
            "📊 Análisis de Datos REAL",
            "💾 Modelos Guardados"
        ])

        st.sidebar.markdown("---")
        st.sidebar.info(
            "Sistema REAL con datos de Kepler, K2 y TESS de la NASA"
        )

        return page

    def render_home(self):
        """Página de inicio"""
        st.title("🪐 NASA Exoplanet Detection AI - SISTEMA REAL")

        col1, col2 = st.columns([2, 1])

        with col1:
            st.markdown("""
            ### Sistema REAL de Detección de Exoplanetas
            - ✅ **Entrenamiento REAL** con datos de la NASA
            - ✅ **Ensemble Stacking** (RF, ET, XGB, LGBM)
            - ✅ **Guardado/Auto-carga** de modelos persistentes
            """)

            st.warning("""
            **ANTES DE CONTINUAR EN COLAB:**
            1. **Sube tus CSV** (`kepler.csv`, `k2.csv`, `tess.csv`)
               a la carpeta `data/raw/` en el explorador de archivos de Colab.
            2. Luego, ve a **'🚀 Entrenar Modelo REAL'**
            """)
        
        with col2:
            st.image("https://www.nasa.gov/sites/default/files/thumbnails/image/kepler_all_planets_art.jpg",
                     caption="Datos REALES de la NASA")

        # Verificar si hay modelo entrenado
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        if os.path.exists(model_path):
            st.success("✅ **Modelo entrenado disponible**")
        else:
            st.warning("⚠️ **No hay modelo entrenado** - Ve a 'Entrenar Modelo REAL'")

    def render_real_training(self):
        """Página de entrenamiento REAL con datos de la NASA"""
        st.title("🚀 Entrenamiento REAL con Datos NASA")

        st.info("""
        El modelo entrenado se guardará automáticamente y estará disponible para clasificación.
        """)

        if st.button("🎯 Iniciar Entrenamiento REAL", type="primary"):
            # Usar st.empty() para mostrar logs del backend si es necesario
            log_placeholder = st.empty() 
            
            with st.spinner("Cargando y procesando datos REALES de la NASA..."):
                try:
                    # Cargar datos reales
                    kepler_df, k2_df, tess_df = self.data_processor.load_real_data()

                    if kepler_df is None or kepler_df.empty:
                        st.error("❌ No se pudieron cargar los datasets o Kepler está vacío. Verifica que los CSV estén en `data/raw/`.")
                        return

                    # Procesar datos
                    kepler_processed = self.data_processor.preprocess_kepler(kepler_df)
                    datasets_to_process = [kepler_processed]
                    if k2_df is not None:
                        k2_processed = self.data_processor.preprocess_k2(k2_df)
                        if k2_processed is not None: datasets_to_process.append(k2_processed)
                    if tess_df is not None:
                        tess_processed = self.data_processor.preprocess_tess(tess_df)
                        if tess_processed is not None: datasets_to_process.append(tess_processed)

                    datasets_to_process = [d for d in datasets_to_process if d is not None and not d.empty]
                    if not datasets_to_process:
                        st.error("❌ No hay datos válidos para procesar")
                        return

                    unified_data = pd.concat(datasets_to_process, ignore_index=True)
                    st.success(f"✅ Datos unificados: {len(unified_data):,} muestras")

                    # Preparar características
                    X, y, feature_names = self.data_processor.prepare_features(unified_data)

                    if X is None:
                        st.error("❌ No se pudieron preparar las características")
                        return

                    # Entrenar modelo
                    st.subheader("🤖 Entrenando Modelo Ensemble...")
                    trained_model = self.model.train(X, y)

                    if trained_model is None:
                        st.error("❌ Error en el entrenamiento")
                        return

                    # Guardar modelo
                    models_dir = os.path.join(PROJECT_ROOT, 'models')
                    model_path = os.path.join(models_dir, 'real_ensemble_model.pkl')
                    processor_path = os.path.join(models_dir, 'data_processor.pkl')

                    model_saved = self.model.save_model(model_path)

                    if model_saved:
                        joblib.dump(self.data_processor, processor_path)
                        st.success("✅ Modelo entrenado y guardado exitosamente!")
                        self.model_trained = True

                        # Mostrar resultados
                        st.subheader("📈 Resultados del Entrenamiento")
                        col1, col2, col3, col4 = st.columns(4)

                        with col1:
                            st.metric("Accuracy", f"{self.model.accuracy:.2%}")
                        with col2:
                            st.metric("Muestras", f"{X.shape[0]:,}")
                        with col3:
                            st.metric("Características", X.shape[1])
                        with col4:
                            st.metric("Algoritmos", "4 Ensemble")

                        # Importancia de características
                        if self.model.feature_importance is not None and feature_names:
                            st.subheader("🔍 Importancia de Características")
                            if len(feature_names) == len(self.model.feature_importance):
                                importance_df = pd.DataFrame({
                                    'Característica': feature_names,
                                    'Importancia': self.model.feature_importance
                                }).sort_values('Importancia', ascending=False)

                                fig = px.bar(
                                    importance_df.head(10),
                                    x='Importancia', y='Característica',
                                    title='Top 10 Características Más Importantes',
                                    orientation='h'
                                )
                                st.plotly_chart(fig, use_container_width=True)
                            else:
                                st.warning("No se pudo graficar la importancia: longitudes no coinciden.")
                    
                    st.balloons()

                except Exception as e:
                    st.error(f"❌ Error durante el entrenamiento: {str(e)}")
                    import traceback
                    st.code(traceback.format_exc())

    def render_real_classification(self):
        """Clasificación con modelo REAL entrenado"""
        st.title("🤖 Clasificación con Modelo REAL")

        # Rutas de archivos
        model_path = os.path.join(PROJECT_ROOT, 'models', 'real_ensemble_model.pkl')
        
        if not os.path.exists(model_path) or not self.model_trained:
            st.warning("⚠️ **No hay modelo entrenado** - Ve a 'Entrenar Modelo REAL' para usar el clasificador.")
            return

        st.success("✅ Modelo REAL y preprocesador cargados exitosamente")

        # Cargar feature names para mostrar en la interfaz
        feature_names = self.data_processor.feature_names
        num_features = len(feature_names)
        st.info(f"🔍 El modelo espera **{num_features}** características: {', '.join(feature_names)}")

        # Definir los nombres amigables y valores por defecto usando un mapa
        feature_map_friendly = {
            'koi_period': ("Período Orbital (días)", 10.0, 0.1, 1000.0),
            'pl_orbper': ("Período Orbital (días)", 10.0, 0.1, 1000.0),
            'period': ("Período Orbital (días)", 10.0, 0.1, 1000.0),
            'koi_duration': ("Duración Tránsito (horas)", 3.0, 0.1, 24.0),
            'koi_depth': ("Profundidad Tránsito (ppm)", 500, 1, 100000),
            'koi_prad': ("Radio Planetario (Radios Tierra)", 2.0, 0.1, 50.0),
            'koi_teq': ("Temperatura Equilibrio (K)", 500, 100, 5000),
            'koi_insol': ("Flujo de Insolación", 100.0, 0.1, 10000.0),
            'koi_steff': ("Temperatura Estelar (K)", 5800, 2000, 15000),
            'koi_slogg': ("Gravedad Estelar (log g)", 4.4, 3.0, 5.5),
            'koi_srad': ("Radio Estelar (Radios Sol)", 1.0, 0.1, 10.0)
        }
        
        input_values = {}

        with st.form("real_classification_form"):
            st.subheader(f"📐 Parámetros del Candidato - {num_features} REQUERIDOS")
            
            # Crear columnas dinámicas (máximo 2 columnas)
            cols = st.columns(min(2, num_features))
            col_index = 0

            # Solo mostrar los inputs para las características que el modelo REAL espera
            for feature_name in feature_names:
                # Buscar la definición de input, si no se encuentra, usar una predeterminada para evitar fallos
                friendly_name, default_value, min_val, max_val = feature_map_friendly.get(
                    feature_name, (feature_name.capitalize().replace('_', ' '), 1.0, 0.0, 100.0)
                )

                with cols[col_index % 2]:
                    # Usar st.text_input si es string, pero para estos datos numéricos, number_input es mejor
                    input_values[feature_name] = st.number_input(
                        friendly_name,
                        min_value=float(min_val),
                        max_value=float(max_val),
                        value=float(default_value),
                        key=f"input_{feature_name}"
                    )
                col_index += 1

            submitted = st.form_submit_button("🚀 Clasificar con Modelo REAL")

        if submitted:
            # 1. Asegurar el ORDEN de las características
            final_features = tuple(input_values[name] for name in feature_names if name in input_values)

            # 2. Verificar la cantidad
            if len(final_features) == num_features:
                self._real_prediction(*final_features)
            else:
                 st.error(f"""
                 ❌ **ERROR DE CARACTERÍSTICAS**
                 **Envías:** {len(final_features)} | **Modelo espera:** {num_features}
                 """)


    def _real_prediction(self, *features):
        """Predicción REAL con el modelo entrenado - BLOQUE DE RESULTADOS FALTANTE CORREGIDO"""
        try:
            # Reutilizar el modelo y el procesador ya cargados en self
            saved_feature_names = self.data_processor.feature_names
            data_processor = self.data_processor

            # VERIFICACIÓN CRÍTICA: ¿Coincide el número de características?
            if len(features) != len(saved_feature_names):
                st.error(f"""
                ❌ **ERROR CRÍTICO - Discrepancia en características**
                **Envías:** {len(features)} | **Modelo espera:** {len(saved_feature_names)}
                """)
                return

            # Crear array de características
            feature_array = np.array([features]).reshape(1, -1)

            # Escalar características (usando el escalador entrenado)
            feature_array_scaled = data_processor.scaler.transform(feature_array)

            # Realizar predicción
            prediction = self.model.model.predict(feature_array_scaled)[0]
            probability = self.model.model.predict_proba(feature_array_scaled)[0, 1]

            # ----------------------------------------------------
            # ✅ BLOQUE DE RESULTADOS AÑADIDO (Error #3 corregido)
            # ----------------------------------------------------
            st.subheader("Resultado de la Clasificación 🧑‍🔬")

            if prediction == 1:
                st.success("✅ **¡El candidato es CLASIFICADO como EXOPLANETA!**")
                st.markdown(f"**Probabilidad de Exoplaneta:** `{probability:.2%}`")
                st.balloons()
            else:
                st.error("❌ **El candidato es CLASIFICADO como FALSO POSITIVO/No Exoplaneta.**")
                st.markdown(f"**Probabilidad de Exoplaneta:** `{probability:.2%}`")
                st.markdown("Recomendación: Necesita más observación o su señal es probable que sea ruido.")
            # ----------------------------------------------------
            
        except Exception as e:
            st.error(f"❌ Error al realizar la predicción: {e}")
            import traceback
            st.code(traceback.format_exc())

    def render_data_analysis(self):
        st.title("📊 Análisis de Datos REAL")
        st.info("Esta sección es para futuras visualizaciones de los datos y el modelo.")
        
    def render_models_saved(self):
        st.title("💾 Modelos Guardados")
        models_dir = os.path.join(PROJECT_ROOT, 'models')
        model_path = os.path.join(models_dir, 'real_ensemble_model.pkl')
        processor_path = os.path.join(models_dir, 'data_processor.pkl')

        st.subheader("Archivos de Persistencia")

        if os.path.exists(model_path):
            st.success(f"✅ Modelo (real_ensemble_model.pkl) encontrado: {os.path.getsize(model_path) / (1024*1024):.2f} MB")
        else:
            st.warning("❌ Modelo no encontrado. Entrena el modelo primero.")
        
        if os.path.exists(processor_path):
            st.success(f"✅ Preprocesador (data_processor.pkl) encontrado: {os.path.getsize(processor_path) / 1024:.2f} KB")
            st.markdown(f"**Características esperadas:** {self.data_processor.feature_names}")
        else:
            st.warning("❌ Preprocesador no encontrado. Entrena el modelo primero.")

    def run(self):
        """Función principal de Streamlit"""
        page = self.render_sidebar()

        if page == "🏠 Inicio":
            self.render_home()
        elif page == "🚀 Entrenar Modelo REAL":
            self.render_real_training()
        elif page == "🤖 Clasificar Exoplanetas":
            self.render_real_classification()
        elif page == "📊 Análisis de Datos REAL":
            self.render_data_analysis()
        elif page == "💾 Modelos Guardados":
            self.render_models_saved()

# ==============================================================================
# 5. EJECUCIÓN DEL SCRIPT Y NGROK (Específico para Colab)
# ==============================================================================

if __name__ == '__main__':
    # 1. Crear el archivo app.py con el contenido del script de Streamlit
    script_content = """
import streamlit as st
from exoplanet_detector import ExoplanetDetectorApp

def main():
    app = ExoplanetDetectorApp()
    app.run()

if __name__ == '__main__':
    main()
"""
    # Guardar el contenido de las clases en un módulo para importarlo
    with open('exoplanet_detector.py', 'w') as f:
        # Aquí se incluye todo el código de las clases (DataProcessor, Model, App) para encapsularlo
        f.write(f"""
{open(__file__).read().split('class ExoplanetDetectorApp:', 1)[0]}
class ExoplanetDetectorApp:{open(__file__).read().split('class ExoplanetDetectorApp:', 1)[1].split('# ==============================================================================\n# 5. EJECUCIÓN DEL SCRIPT Y NGROK', 1)[0]}
""")

    # Guardar el script de ejecución principal de Streamlit
    with open('app.py', 'w') as f:
        f.write(script_content)

    # 2. Ejecutar Streamlit en segundo plano
    print("🚀 Iniciando Streamlit en segundo plano...")
    subprocess.Popen(['streamlit', 'run', 'app.py'])

    # 3. Configurar ngrok
    print("🌐 Iniciando ngrok...")
    # Asegúrate de tener tu token de ngrok si la versión más reciente lo requiere
    # ngrok.set_auth_token("TU_NGROK_TOKEN_AQUI") 
    
    # Esperar un momento para que Streamlit se inicie completamente (puerto 8501 por defecto)
    time.sleep(5)
    
    # Intentar establecer el túnel
    try:
        public_url = ngrok.connect(8501)
        print(f"✨ ¡STREAMLIT EJECUTÁNDOSE EN COLAB! ✨")
        print(f"🔗 URL Pública (clic aquí): {public_url}")
        st.code(str(public_url)) # Mostrar la URL en la salida del notebook
    except Exception as e:
        print(f"❌ Error al conectar con ngrok: {e}")
        print("Asegúrate de que streamlit esté funcionando en el puerto 8501 y que ngrok esté configurado.")